In [1]:
import numpy as np
from modules import bayesianL
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy


In [68]:
1/(0.01*max(datasigmaplot*np.sqrt(10.)))

1937.064485168971

In [2]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[0. 1. 0.] [0. 0. 1.] [1. 0. 0.]


In [238]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e 
#ritorno la lista dei nuovi dati 'simmetrizzati' in gplot e sd
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)

sd=np.array([dataplot, 1200*datasigmaplot*np.sqrt(10.)])

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [239]:
#Calcolo il fit bayesiano con glot e sd come test
N=100
mN, SN, y_infer, sy_infer, conta=\
bayesianL.bayesianpol(gplot, sd, 8, N, 0.0001, gplot,bethapar=10000, ifprint= True, ifwarning= True, nLbp=4)

parametri ottimali [  0.54208759  -6.54226469   0.19954694  -0.39659824   0.19867995
  -0.20297443  13.18662261  -1.39707857   2.60130693  -1.69423215
   1.5693718  -10.51641487   3.87686937  -5.83597829   5.03738672
  -4.67081363   0.06709774  -5.54428131   5.66768204  -7.13523505
   6.92023584   2.8627652    4.12227346  -2.28401574   5.17340085
  -5.42655833   0.45044203  -1.28399332   0.26475836  -1.78619443
   2.14745316  -0.88697689   0.02755287  -0.02357334   0.20318643
  -0.33797438]
numero di armoniche cubiche 36
valore a kmin inferito 0.5020885856364139 bias 0.5420875921357016 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 21032128682947.03


In [240]:
#Plotto il fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot1, datasigmaplot1=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,0*datasigmaplot*np.sqrt(10.), fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [242]:
#Calcolo il best fit su gplot e sd con 50 dati
N=10
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesianL.bestfit(gplot, sd, N, gplot, ifbetha=True, ifprintbestfit= False, ifprintfinal= True, nLbf=1)
#Plotto il best fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot1, datasigmaplot1=bayesianL.datiplot(C, dic, dics)


%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.')

parametri ottimali [   0.84046531  -35.07338361    1.68082684  217.81234375  -30.90955687
 -444.69557946  172.63758768 -128.08388436 -281.46148448]
numero di armoniche cubiche 9
valore a kmin inferito 0.6981862104268622 bias 0.8404653101012516 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 9.354063276999903e-13
grado ottimale 10 grado massimo tentato 28
numero di polinomi nella base ottimale:  9 numero di dati 10
best alpha 6.506561998270968e-06
best betha 6301019.151450007
numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [243]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(range(2,(len(log_evidence_vP)+1)*2, 2), log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
#Calcolo il best fit includendo sempre piu' punti 
predbl=[]
spredbl=[]
Npointsbl=[]
for i in range(5):
    predb=[]
    spredb=[]
    Npointsb=[]
    for N in range(11, 56):
        
        mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
        bayesianL.bestfit(gplot, sd, N, gplot, ifbetha=True, ifprintbestfit=False, ifprintfinal=False, nLbf=i)
        Npointsb.append(np.linalg.norm(gplot[N]))
        if (N-11)%(40//5)==0: print(int((N-11)/40*100), '%')
        predb.append(mN[0])
        spredb.append(SN[0,0])
    predbl.append(predb)
    spredbl.append(spredb)
    Npointsbl.append(Npointsb)

0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %


/Users/enricodrigo/opt/anaconda3/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


20 %
40 %
60 %
80 %
100 %


In [246]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


#ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')
#
#ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
#ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
#    
#ax[0].set_xlabel('$k_{max} (nm^{-1})$')
#ax[0].set_ylabel('prediction at k=0')
#ax[0].legend()
ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')
L=[0,[0,4],[0,4,6], [0,4,6,8], [0,4,6,8,10]]
for i in range(2):
    ax[0].errorbar(np.array(Npointsbl[i]), np.array(predbl[i]), np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
ax[0].fill_between(np.array(Npointsbl[0]), np.ones(len(Npointsbl[0]))*0.816 + np.ones(len(Npointsbl[0]))*0.021, np.ones(len(Npointsbl[0]))*0.816 - np.ones(len(Npointsbl[0]))*0.021, label='GK result', alpha=0.3)    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.', label='3375 water molecules at 300k 30ns') 
ax[1].fill_between(np.linalg.norm(gplot, axis=1), np.ones(len(gplot))*0.816 + np.ones(len(gplot))*0.021, np.ones(len(gplot))*0.816 - np.ones(len(gplot))*0.021, label='GK result', alpha=0.3)
ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')
#plt.savefig('nlttbetha.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'k ($nm^{-1}$)')

In [72]:
#Calcolo il best fit includendo sempre piu' punti 
predbl=[]
spredbl=[]
Npointsbl=[]
for i in range(5):
    predb=[]
    spredb=[]
    Npointsb=[]
    for N in range(5, 56):
        mN, SN,log_evidence_vP=\
        bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=False, nLbmp=i)
        Npointsb.append(np.linalg.norm(gplot[N]))
        if (N-5)%(50//5)==0: print(int((N-5)/50*100), '%')
        predb.append(mN)
        spredb.append(SN)
    predbl.append(predb)
    spredbl.append(spredb)
    Npointsbl.append(Npointsb)

0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %


In [73]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


#ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')
#
#ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
#ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
#    
#ax[0].set_xlabel('$k_{max} (nm^{-1})$')
#ax[0].set_ylabel('prediction at k=0')
#ax[0].legend()
ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')
L=[0,[0,4],[0,4,6], [0,4,6,8], [0,4,6,8,10]]
for i in range(5):
    ax[0].errorbar(np.array(Npointsbl[i]), np.array(predbl[i]), np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
#ax[0].plot(np.array(Npointsbl[1]), np.ones(len(Npointsbl[1]))*0.816, label='GK result')    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.', label='3375 water molecules at 300k 30ns') 

ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'k ($nm^{-1}$)')